# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
H. Jiang  ->  H. Jiang  |  ['H. Jiang']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
F. Xu  ->  F. Xu  |  ['F. Xu']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
M. Kürster  ->  M. Kürster  |  ['M. Kürster']


H. Beuther  ->  H. Beuther  |  ['H. Beuther']
C. Gieser  ->  C. Gieser  |  ['C. Gieser']
S. Jiao  ->  S. Jiao  |  ['S. Jiao']
M. Wells  ->  M. Wells  |  ['M. Wells']
J. Wolf  ->  D. J. Wolf  |  ['J. Wolf']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
Arxiv has 75 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.24877


extracting tarball to tmp_2510.24877...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.24886


extracting tarball to tmp_2510.24886... done.
  0: tmp_2510.24886/natbib.tex, 96 lines
  1: tmp_2510.24886/aassymbols.tex, 579 lines
  2: tmp_2510.24886/natnotes.tex, 332 lines
  3: tmp_2510.24886/main.tex, 869 lines
Retrieving document from  https://arxiv.org/e-print/2510.25024


...

/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure mara_sep_combi_LS10
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure mara_sep_combi_LS10 as tmp_2510.25589/./plots/mara_sep_combi_LS10.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure logn_logs_fin_fit
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure logn_logs_fin_fit as tmp_2510.25589/./plots/logn_logs_fin_fit.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure 4LAC_alpha
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure 4LAC_alpha as tmp_2510.25589/./plots/4LAC_alpha.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 198 bibliographic references in tmp_2510.25589/main.bbl.
Issues with the citations
syntax error in line 277: '=' expected


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{HI4PI Collaboration}(2016)}]{hi4picollaboration:2016}{HI4PI Collaboration}. 2016, \aap, 594, A116
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.25078-b31b1b.svg)](https://arxiv.org/abs/2510.25078) | **Magnetic Fields in Massive Star-forming Regions (MagMaR). VI. Magnetic Field Dragging in the Filamentary High-mass Star-forming Region G35.20--0.74N due to Gravity**  |
|| J. Hwang, et al. -- incl., <mark>F. Xu</mark>, <mark>H. Beuther</mark> |
|*Appeared on*| *2025-10-30*|
|*Comments*| *submitted to AJ*|
|**Abstract**|            We investigate the magnetic field orientation and strength in the massive star-forming region G35.20-0.74N (G35), using polarized dust emission data obtained with the Atacama Large Millimeter/submillimeter Array (ALMA) as part of the Magnetic fields in Massive star-forming Regions (MagMaR) survey. The G35 region shows a filamentary structure (a length of $\sim$0.1 pc) with six bright cores located along the filament's long axis. Magnetic field strengths across the G35 region range from 0.2 to 4.4 mG with a mean value of 0.8 $\pm$ 0.4 mG. The mass-to-flux ratio ($\lambda$) varies from 0.1 to 6.0 the critical value. The highest values are found locally around cores, whereas the remains of the filament are subcritical. A H$^{13}$CO$^+$ (3--2) velocity gradient of 29 km s$^{-1}$ pc$^{-1}$ is evident along the filament's long axis, aligned with the magnetic field direction. At larger scales ($\sim$0.1 pc), the magnetic field lines appear roughly perpendicular to the filament's long axis, in contrast to the smaller-scale structure ($\sim$0.003 pc) traced by ALMA. The magnetic field lines could be dragged along the filament as a result of the gas motion induced by the gravitational potential of the filament. Six cores in the filament have similar spacings between 0.02--0.04 pc. The initial filament fragmentation could have produced a core spacing of 0.06 pc, following filament fragmentation theory, and the current core spacing is the result of cores comoving with the gas along the filament. This core migration could occur in a few 10$^4$ years, consistent with high-mass star formation time scales.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.25538-b31b1b.svg)](https://arxiv.org/abs/2510.25538) | **Filamentary accretion flows in high-mass star-forming clouds**  |
|| J.-E. Schneider, et al. -- incl., <mark>H. Beuther</mark>, <mark>C. Gieser</mark>, <mark>S. Jiao</mark>, <mark>M. Wells</mark> |
|*Appeared on*| *2025-10-30*|
|*Comments*| **|
|**Abstract**|            The scope of this paper is focused on the question if gas is transported to dense clumps inside high-mass star-forming regions through filamentary structures on a scale of several parsecs to the sub-parsec scale. We quantify the gas flows from a scale of up to several parsecs down to the sub-parsec scale along filamentary structures. In this work the accretion flow mechanisms based on gas kinematic data in the three high-mass star-forming regions G75.78, IRAS21078+5211 and NGC7538 are studied with data obtained from the IRAM 30 m telescope. The analysis is carried out using the surface density derived from 1.2 mm continuum emission and velocity differences estimated from HCO+ and H13CO+ molecular line data.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.25589-b31b1b.svg)](https://arxiv.org/abs/2510.25589) | **BlazEr1: The eROSITA Blazar Catalog. Blazars and Blazar Candidates in the First eROSITA Survey**  |
|| S. Haemmerich, et al. -- incl., <mark>J. Wolf</mark>, <mark>S. Belladitta</mark> |
|*Appeared on*| *2025-10-30*|
|*Comments*| *39 pages, 28 figures, submitted to A&A. Pre-referee version, comments are appreciated*|
|**Abstract**|            Aims. eROSITA on board of the Spectrum Roentgen Gamma (SRG) spacecraft performed its first X-ray all-sky survey (eRASS1) between December 2019 and June 2020. It detected about 930000 sources, providing us with an unprecedented opportunity for a detailed blazar census. We present the properties of blazars and blazar candidates in eRASS1 and the compilation of the eROSITA blazar catalog. Methods. We compile a list of blazar and blazar candidates from the literature and match it with the eRASS1 catalog, creating the Blazars in eRASS1 (BlazEr1) catalog. For sources with more than 50 counts we obtain their X-ray spectral properties. We compile multiwavelength data from the radio to the gamma-ray regimes for all sources, including multiwavelength spectral indices and redshifts. The full catalog is available online. Results. We present the BlazEr1 catalog, containing 5865 sources, of which 2106 are associated with confirmed blazars. For 3668 sources, eROSITA provides the first X-ray data. The contamination from non-blazar sources of the entire sample is less than 11%. Most candidates exhibit properties typical for blazars. We present properties of the entire X-ray detected blazar population, including the distributions of X-ray luminosities and photon indices, multiwavelength properties, and the blazar log N-log S distribution. Our catalog provides follow up targets, such as potential MeV and TeV blazars. Conclusions. The BlazEr1 catalog provides a compilation of X-ray detected blazars and blazar candidates. The catalog serves as a starting point for exploiting further eROSITA surveys using the same methodology, enabling us to study the X-ray variability and a large number of spectral energy distributions of blazars in the future.         |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.24886-b31b1b.svg)](https://arxiv.org/abs/2510.24886) | **Dissecting the mass quenching in TNG50: Galaxy size determines the quenching mode**  |
|| <mark>H. Jiang</mark>, et al. |
|*Appeared on*| *2025-10-30*|
|*Comments*| *15 pages, 7 figures, ApJ accepted*|
|**Abstract**|            The diminishing of star formation is accompanied by size differentiating, as quiescent galaxies are more compact than star-forming galaxies at fixed stellar mass. In order to understand how galaxy quenching is related to galaxy sizes, we performed a demographic study of 46 massive quiescent central galaxies with stellar mass from $10^{10.5}\rm M_\odot$ to $10^{11}\rm M_\odot$ in the TNG50 simulation. We found that, in addition to the triggering active galactic nucleus (AGN) feedback, galaxy size is also a major determinant of the quenching process, as small and compact galaxies are immediately quenched by the kinetic AGN feedback, while galaxies with large sizes are still active until strangulated by the cutoff of new gas replenishment. Further spatially resolved inspection reveals that this short and intense kinetic AGN feedback can only suppress the star formation within 1-2 kpc, resulting in this size-dependent effect of quenching. We also identify a long-term effect of a few Gyr timescale that the gas inflow rate is progressively suppressed after triggering kinetic feedback, which appears to effectively quench large galaxies entirely. We conclude that kinetic AGN feedback has two key roles in quenching: a short-term, intense effect that quenches the central 2 kpc region, and a long-term effect that suppresses the gas inflow rate and further quenches the entire galaxy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.25097-b31b1b.svg)](https://arxiv.org/abs/2510.25097) | **A Systematic Search for Gaseous Debris Disks in DESI Early Data Release White Dwarfs**  |
|| Z. Ma, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-10-30*|
|*Comments*| *9 pages, 3 figures, 1 table; AJ accepted*|
|**Abstract**|            Detecting gaseous debris disks around white dwarfs offers a unique window into the ultimate fate of planetary systems and the composition of accreted planetary material. Here we present a systematic search for such disks through the Ca II infrared triplet using the Dark Energy Spectroscopic Instrument (DESI) Early Data Release. From a parent sample of 2706 spectroscopically confirmed white dwarfs, we identify 22 candidate systems showing tentative emission-line features, which corresponds to a raw occurrence rate of 0.81%, more than ten times higher than previous estimates. The detected emission lines are predominantly weak and require confirmation by follow-up observations. Three of these candidates also exhibit infrared excess in WISE photometry, suggesting a possible coexistence of gas and dust. However, the high candidate rate indicates that most are likely false positives due to telluric residuals or unresolved binaries. This work demonstrates the potential of DESI spectra for blind searches of rare circumstellar phenomena. The recently released DESI DR1, with its substantially larger spectroscopic sample, will enable searches for more gaseous disks and provide better insights into their occurrence and nature.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.25247-b31b1b.svg)](https://arxiv.org/abs/2510.25247) | **Evolution of accretion disc-corona in the TDE Candidate AT 2019avd**  |
|| H. Xu, X. Cao, <mark>Y. Wang</mark>, A. A. Zdziarski |
|*Appeared on*| *2025-10-30*|
|*Comments*| *10 pages,6 figures,1 table*|
|**Abstract**|            X-ray observations of the tidal disruption event (TDE) candidate AT 2019avd show drastic variabilities in flux and spectral shape over hundreds of days, providing clues on the accretion disc-corona evolution. We utilize a disc-corona model, in which a fraction of the gravitational energy released in the disc is transported into the hot corona above/below. Some soft photons emitted from the disc are upscattered to X-ray photons by the hot electrons in the optically thin corona. By fitting the NICER observations of AT 2019avd during epochs when the spectra exhibit significant hardening, we derive the evolution of the mass accretion rate, $\dot{m}$, and the coronal energy fraction, $f$. Our results show that $f$ decreases with increasing $\dot{m}$, which is qualitatively consistent with that observed in active galactic nuclei (AGNs), while the slope of this source, $f\propto \dot{m}^{-0.30}$, is much shallower than that of AGNs. We also find that the non-thermal X-ray spectrum in this source is significantly softer than those typically seen in AGNs and black-hole X-ray binaries. We argue that these quantitative differences can be a powerful diagnostic of the underlying magnetic turbulence, which may imply a stronger magnetic field within the TDE accretion disc than that in typical AGNs. It is also found that the evolution of the fitted neutral hydrogen column density follows a similar pattern to that of the accretion rate evolution, which may reflect the accumulation of absorbing material originating from the inflowing streams of stellar debris and/or other related sources.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.25024-b31b1b.svg)](https://arxiv.org/abs/2510.25024) | **Where Galaxies Go to Die: The Environments of Massive Quiescent Galaxies at $3<z<5$**  |
|| I. McConachie, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-10-30*|
|*Comments*| *Submitted to ApJ; 27 pages, 14 figures, 4 tables. Comments welcome! Animation and interactive figure hosted on Google Drive at this https URL*|
|**Abstract**|            At low redshift, massive quiescent galaxies (MQGs) are most frequently found in massive, rich galaxy clusters, but at high redshift the trend is less clear. Here, we present spectroscopic evidence of the effects of environment on the formation and assembly of high-redshift MQGs. We identify 25 (5) $\log (M_*/\mathrm{M_\odot}\geq10.5$ ($10.0\leq\log (M_*/\mathrm{M_\odot}<10.5$) spectroscopically-confirmed quiescent galaxies in the UDS and EGS fields at $3<z<5$ with NIRSpec PRISM spectroscopy from RUBIES and other public JWST NIRSpec programs. We measure the density contrast in these fields by applying a Monte Carlo Voronoi Tesselation density mapping technique to photometric and spectroscopic redshifts of $m_\mathrm{F444W}<27.5$ sources. We robustly detect 12 massive overdense peaks with $\log (M_\mathrm{Peak}/\mathrm{M_\odot})\geq13$ and six extended massive protoclusters ($\log (M_\mathrm{Struct}/\mathrm{M_\odot})\geq13.85$). We observe that MQGs are preferentially found in these massive peaks and within these massive structures: $\approx50\%$ of MQGs are found in massive peaks, compared to $\approx20\%$ of massive star forming galaxies (MSFGs) and $\approx15\%$ of the overall spectroscopically-confirmed population. We also find an apparent dependence on both quiescent galaxy mass and environment, with $75\%$ of the most massive ($\log (M_*/\mathrm{M_\odot}\geq10.75$) residing inside overdense peaks. We compare the star formation histories (SFHs) of the MQGs with the high-redshift galaxy stellar mass function from observations and simulated quiescent galaxies at $z>5$, finding that the masses from the inferred MQG SFHs regularly exceed either observed or simulated high-redshift galaxies, which suggests indicates that mergers and ex-situ star formation play a key role in the mass assembly of MQGs in overdense environments.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.25334-b31b1b.svg)](https://arxiv.org/abs/2510.25334) | **The CARMENES search for exoplanets around M dwarfs : Understanding the wavelength dependence of radial velocity measurements**  |
|| S. V. Jeffers, et al. -- incl., <mark>M. Kürster</mark> |
|*Appeared on*| *2025-10-30*|
|*Comments*| *published in A&A*|
|**Abstract**|            Context. Current exoplanet surveys are focused on detecting small exoplanets orbiting in the liquid-water habitable zones of their host stars. Despite the recent significant advancements in instrumental developments, the current limitation in detecting these exoplanets is the intrinsic variability of the host star itself. Aims. Our aim is to use the full CARMENES guaranteed time observations (GTO) data set spanning more than 8 years of observations of over 350 stars to investigate the wavelength dependence of high-precision radial velocities (RV), as stellar activity features should exhibit a wavelength dependence while the RV variation due to an orbiting planet will be wavelength independent. Methods. We use the chromatic index (CRX) to quantify the slope of the measured RVs as a function of logarithmic wavelength. We investigate the dependence of the CRX in the full CARMENES GTO sample on 24 stellar activity indices in the visible and near-infrared channels of the CARMENES spectrograph and each star's stellar parameters. We also present an updated convective turnover time scaling for the calculation of the stellar Rossby number for M dwarfs. Results. Our results show that approximately 17\% of GTO stars show a strong or a moderate correlation between CRX and RV. We can improve the measured RVs by a factor of up to nearly 4 in rms by subtracting the RV predicted by the CRX-RV correlation from the measured RVs. Mid M dwarfs with moderate rotational velocities, moderate CRX-gradients and quasi-stable activity features have the best rms improvement factors. Conclusions. We conclude that the CRX is a powerful diagnostic in mitigation of stellar activity and the search for low mass rocky planets.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.25078/./G35_angdisp_volden_veldisp.png', 'tmp_2510.25078/./G35_magnetic_field_orientation_v6.png', 'tmp_2510.25078/./G35_mag_fila_orientations_v3.png']
copying  tmp_2510.25078/./G35_angdisp_volden_veldisp.png to _build/html/
copying  tmp_2510.25078/./G35_magnetic_field_orientation_v6.png to _build/html/
copying  tmp_2510.25078/./G35_mag_fila_orientations_v3.png to _build/html/
exported in  _build/html/2510.25078.md
    + _build/html/tmp_2510.25078/./G35_angdisp_volden_veldisp.png
    + _build/html/tmp_2510.25078/./G35_magnetic_field_orientation_v6.png
    + _build/html/tmp_2510.25078/./G35_mag_fila_orientations_v3.png
found figures ['tmp_2510.25538/./Images/dust_cont_allsources.png', 'tmp_2510.25538/./Images/mom1_hco+_allsources.png', 'tmp_2510.25538/./Images/mom1_h13co+_allsources.png']
copying  tmp_2510.25538/./Images/dust_cont_allsources.png to _build/html/
copying  tmp_2510.25538/./Images/mom1_hco+_allsources.png to _build/html/
copying  tmp_2510.25538/./Im

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Magnetic Fields in Massive Star-forming Regions (MagMaR). VI. Magnetic Field Dragging in the Filamentary High-mass Star-forming Region G35.20--0.74N due to Gravity

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.25078-b31b1b.svg)](https://arxiv.org/abs/2510.25078)<mark>Appeared on: 2025-10-30</mark> -  _submitted to AJ_

</div>
<div id="authors">

J. Hwang, et al. -- incl., <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** We investigate the magnetic field orientation and strength in the massive star-forming region G35.20-0.74N (G35), using polarized dust emission data obtained with the Atacama Large Millimeter/submillimeter Array (ALMA) as part of the Magnetic fields in Massive star-forming Regions (MagMaR) survey. The G35 region shows a filamentary structure (a length of $\sim$ 0.1 pc) with six bright cores located along the filament's long axis. Magnetic field strengths across the G35 region range from 0.2 to 4.4 mG with a mean value of 0.8 $\pm$ 0.4 mG. The mass-to-flux ratio ( $\lambda$ ) varies from 0.1 to 6.0 the critical value. The highest values are found locally around cores, whereas the remains of the filament are subcritical.A H $^{13}$ CO $^+$ (3--2) velocity gradient of 29 km s $^{-1}$ pc $^{-1}$ is evident along the filament's long axis, aligned with the magnetic field direction. At larger scales ( $\sim$ 0.1 pc), the magnetic field lines appear roughly perpendicular to the filament's long axis, in contrast to the smaller-scale structure ( $\sim$ 0.003 pc) traced by ALMA.The magnetic field lines could be dragged along the filament as a result of the gas motion induced by the gravitational potential of the filament. Six cores in the filament have similar spacings between 0.02--0.04 pc. The initial filament fragmentation could have produced a core spacing of 0.06 pc, following filament fragmentation theory, and the current core spacing is the result of cores comoving with the gas along the filament. This core migration could occur in a few 10 $^4$ years, consistent with high-mass star formation time scales.

</div>

<div id="div_fig1">

<img src="tmp_2510.25078/./G35_angdisp_volden_veldisp.png" alt="Fig3" width="100%"/>

**Figure 3. -** The maps of polarization angle dispersion (Left), volume density (Middle), and non-thermal velocity dispersion of the non-thermal component of H$^{13}$CO$^+$(Right) in G35. Black contours show the flux density of G35 at 10, 20, 30, and 40 $\times$$\delta I$ levels. The region within 10 $\times$$\delta I$ level is comparable to the filament with the width of 0.015 estimated in Section \ref{sec:vol} along the black skeleton shown in Figure \ref{fig:fila}.
 (*fig:poldisp*)

</div>
<div id="div_fig2">

<img src="tmp_2510.25078/./G35_magnetic_field_orientation_v6.png" alt="Fig1" width="100%"/>

**Figure 1. -** Map of the magnetic field orientation obtained from dust polarization observations using ALMA in G35. The background image is the intensity (Stokes $I$) at Band 6 frequency, $\sim250$ GHz. The white segments show the magnetic field orientatio,n which is the polarization angle rotated by 90 degrees, and is plotted roughly per independent beam. The triangles are the positions of cores found in previous ALMA dust continuum observations with a higher angular resolution  dthan ours \citep{Zhang2022} The core names are labeled based on previous studies \citep{Zhang2022, Sanchez2013}. The physical scale and beam size are shown in the bottom left and right corners, respectively.
 (*fig:magori*)

</div>
<div id="div_fig3">

<img src="tmp_2510.25078/./G35_mag_fila_orientations_v3.png" alt="Fig2" width="100%"/>

**Figure 2. -** (Left) The skeleton of the filament in G35. Background image is the same as in Figure \ref{fig:magori}. White contours show the flux density of G35 at 3, 10, 20, 30, and 40 $\times$$\delta I$ levels. (Right) Angle difference between the main skeleton and magnetic field orientations.
 (*fig:fila*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.25078"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$</div>



<div id="title">

# Filamentary accretion flows in high-mass star-forming clouds

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.25538-b31b1b.svg)](https://arxiv.org/abs/2510.25538)<mark>Appeared on: 2025-10-30</mark> - 

</div>
<div id="authors">

J.-E. Schneider, et al. -- incl., <mark>H. Beuther</mark>, <mark>C. Gieser</mark>, <mark>S. Jiao</mark>, <mark>M. Wells</mark>

</div>
<div id="abstract">

**Abstract:** Filamentary accretion-flows as gas-funneling mechanisms are a key aspect in high-mass star formation research. The kinematic properties along these structures are of particular interest. The scope of this paper is focused on the question if gas is transported to dense clumps inside high-mass star-forming regions through filamentary structures on a scale of several parsecs to the sub-parsec scale. We quantify the gas flows from a scale of up to several parsecs down to the sub-parsec scale along filamentary structures. In this work the accretion flow mechanisms based on gas kinematic data in the three high-mass star-forming regions G75.78, IRAS21078+5211 and NGC7538 are studied with data obtained from the IRAM 30 m telescope. The analysis is carried out using the surface density derived from 1.2 mm continuum emission and velocity differences estimated from HCO $^+$ ( $1-0$ ) and H $^{13}$ CO $^+$ ( $1-0$ ) molecular line data. The mass flow-behavior of the gas in the vicinity of high-mass star-forming clumps shows characteristic dynamical patterns, such as an increased mass flow rate towards the clumps. We assume the velocity differences to originate from filamentary-gas infall onto the high-mass star-forming clumps; the inclination of the filament-structures along the line of sight is however unknown. Nevertheless, using the velocity differences and mass surface densities, we can estimate mean flow rates along the filamentary structures with respect to the line of sight and towards clumps. We quantify the flow rates towards the clumps in a range from about $10^{-3} M_{\odot} $ yr $^{-1}$ to $10^{-5} M_{\odot}$ yr $^{-1}$ , inferred from clump-centered polar plots. Slight variations of the flow rates along the filamentary structures may be caused by overdensities and velocity gradients along the filaments. While the initial studies presented here already reveal interesting results such as an increasing mass flow rate towards clumps, the properties of filamentary gas flows from large to small spatial scales, as well as potential variations over the evolutionary sequence, are subject to future studies.

</div>

<div id="div_fig1">

<img src="tmp_2510.25538/./Images/dust_cont_allsources.png" alt="Fig30" width="100%"/>

**Figure 30. -** Naming and measuring pattern for the three sourcesTop row: NIKA2 1.2 mm dust continuum maps (see  ([Beuther, Gieser and Soler 2024]()) ) of the star-forming regions showing the clump and filament designations. The red and magenta markers indicate the data points used for the flow rate estimation in Section \ref{sec:fila-flows}. Triangular magenta markers indicate a separate section along a structure. Black contours outline the 1.2 mm continuum, ranging from 3$\sigma$ to 39$\sigma$ in steps of 3$\sigma$(the RMS-values are presented in Table \ref{tab:dust_hii_rms}). The red contours in the upper panels mark the peak-positions of the continuum emission from 20\% to 100\% in steps of 20\%. Designations used in the following refer to clumps and filament-structures, filaments are marked by an "F" in the designation; "CF" in G75 stands for "central filament", "CC" in IRAS21078 refers to "central clump" while the clumps in NGC7538 are the known objects IRS9, IRS1 and S (e.g.  ([ and Beuther 2020]()) ). G75 S1 is the object G75.78+0.34. We present the clump-designations together with their coordinates in Table \ref{tab:clumps}. Bottom row: The same dust-maps with red contour lines outlining the \HII-regions at 1.4 GHz (21 cm) ( ([Condon, Cotton and Greisen 1998]()) ); the contours range from 10\% to 100\% of the peak emission in steps of 10\%(see Table \ref{tab:dust_hii_rms} for reference). The positions of the two main exciting sources of the \HII-region in NGC7538, NGC7538 IRS5 and NGC7538 IRS6, are marked in the lower right panel ( ([Puga, Marín-Franch and Najarro 2010]()) ). (*fig:dusttags*)

</div>
<div id="div_fig2">

<img src="tmp_2510.25538/./Images/mom1_hco+_allsources.png" alt="Fig31" width="100%"/>

**Figure 31. -** 1st moment maps HCO$^+$($1-0$)First moment (intensity-weighted peak-velocity) maps for the HCO$^+$($1-0$)-line. The beam size is \SI{27}{$\arcsec$ond}(lower left corner). The sources are labeled in each panel. The scale bar indicates the length of 1 parsec in each panel. Contour lines show the 1.2 mm dust continuum ranging from 3$\sigma$ to 39$\sigma$ in 3$\sigma$-steps. (*fig:1stmoment_hco+*)

</div>
<div id="div_fig3">

<img src="tmp_2510.25538/./Images/mom1_h13co+_allsources.png" alt="Fig32" width="100%"/>

**Figure 32. -** 1st moment maps H13CO$^+$($1-0$)As for Fig. \ref{fig:1stmoment_hco+} but for H$^{13}$CO$^+$($1-0$)-transition. (*fig:1stmoment_h13co+*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.25538"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\makeLineNumber}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# BlazEr1: The eROSITA Blazar Catalog: Blazars and Blazar Candidates in the First eROSITA Survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.25589-b31b1b.svg)](https://arxiv.org/abs/2510.25589)<mark>Appeared on: 2025-10-30</mark> -  _39 pages, 28 figures, submitted to A&A. Pre-referee version, comments are appreciated_

</div>
<div id="authors">

S. Hämmerich, et al. -- incl., <mark>J. Wolf</mark>, <mark>S. Belladitta</mark>

</div>
<div id="abstract">

**Abstract:**            Aims. eROSITA on board of the Spectrum Roentgen Gamma (SRG) spacecraft performed its first X-ray all-sky survey (eRASS1) between December 2019 and June 2020. It detected about 930000 sources, providing us with an unprecedented opportunity for a detailed blazar census. We present the properties of blazars and blazar candidates in eRASS1 and the compilation of the eROSITA blazar catalog. Methods. We compile a list of blazar and blazar candidates from the literature and match it with the eRASS1 catalog, creating the Blazars in eRASS1 (BlazEr1) catalog. For sources with more than 50 counts we obtain their X-ray spectral properties. We compile multiwavelength data from the radio to the gamma-ray regimes for all sources, including multiwavelength spectral indices and redshifts. The full catalog is available online. Results. We present the BlazEr1 catalog, containing 5865 sources, of which 2106 are associated with confirmed blazars. For 3668 sources, eROSITA provides the first X-ray data. The contamination from non-blazar sources of the entire sample is less than 11%. Most candidates exhibit properties typical for blazars. We present properties of the entire X-ray detected blazar population, including the distributions of X-ray luminosities and photon indices, multiwavelength properties, and the blazar log N-log S distribution. Our catalog provides follow up targets, such as potential MeV and TeV blazars. Conclusions. The BlazEr1 catalog provides a compilation of X-ray detected blazars and blazar candidates. The catalog serves as a starting point for exploiting further eROSITA surveys using the same methodology, enabling us to study the X-ray variability and a large number of spectral energy distributions of blazars in the future.         

</div>

<div id="div_fig1">

<img src="tmp_2510.25589/./plots/blazar_erass1_map_paper.png" alt="Fig2" width="100%"/>

**Figure 2. -** Positions of the BlazEr1 sources in Galactic coordinates. The
  different colors and symbols indicate the blazar types in the
  catalog. Candidates, especially _BCUC_s, dominate the sample.
  The overdensity along $l\sim 240◦$ coincides with the footprint
  of BROS. Confirmed blazars are displayed by filled symbols,
  candidates are shown in the same color without filling. All
  following figures will adapt the same color and symbol scheme.  (*fig:blazar_erass1_map_paper*)

</div>
<div id="div_fig2">

<img src="tmp_2510.25589/./plots/gamma_master_gaus.png" alt="Fig11" width="100%"/>

**Figure 11. -** Probability distribution of photon indices for
	different types of _eROSITA_ observed blazars and blazar candidates.
	The distribution takes into account the uncertainties of
individual measurements of the photon index. (*fig:gamma_master_gaus*)

</div>
<div id="div_fig3">

<img src="tmp_2510.25589/./plots/mara_combiplotfin.png" alt="Fig24" width="100%"/>

**Figure 24. -** Different photometric spaces combining optical, infrared and X-ray data, similar to
  \citet[][; Fig. 18]{salvato:2022} for point sources detected in eFEDS, using the
  color scheme introduced in
  Fig. \ref{fig:blazar_erass1_map_paper}. Solid gray lines
  show positions usually occupied by quasars, whereas the gray dotted
  lines corresponded to Seyfert 2 galaxies. The dashed black lines in
  the upper panels distinguish between Galactic and
  extragalactic sources. (*fig:mara_combiplotfin*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.25589"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

111  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
